In [ ]:
# Import stuff
from pandas import DataFrame, read_csv
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl import GLM, Split, Merge, Cluster
from nipype.interfaces.fsl.maths import MathsCommand

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/data/perlman/moochie/user_data/CamachoCat/ChEC/fmri_proc'
templates_dir = '/data/perlman/moochie/user_data/CamachoCat/Aggregate_anats/templates'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevelMELM'
workflow_dir = analysis_home + '/workflows'
affect_ratings = analysis_home + '/misc/ratings_20200427.csv'

gm_mask = templates_dir + '/lcbd_template_2mm_gm.nii.gz'

subject_info = read_csv(analysis_home + '/misc/subjectinfo.csv', index_col=None)
subjects_list = subject_info['SubjID'].tolist()
#subjects_list = ['1024','1046','1047']

# data collection specs
TR = 0.8 #in seconds
duration= 1284

In [ ]:
# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Pull files
#file_template = {'preproc_func': preproc_dir + '/fully_processed_func/{subjid}/lomo_func.nii.gz'}
file_template = {'preproc_func': preproc_dir + '/precensoring_func/{subjid}/resids_filt.nii.gz'}
selectfiles = Node(SelectFiles(file_template), name='selectfiles')

# Sink data of interest 
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = firstlevel_dir,
                        container = firstlevel_dir,
                        substitutions = substitutions), 
                name='datasink')

In [ ]:
## Timing handling nodes
def affectiveTiming(ratings_file,TR, duration):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    import numpy as np
    from pandas import read_csv
    from os.path import abspath 
    import matplotlib.pyplot as plt
    
    def hrf(time):
        from scipy.stats import gamma
        peak = gamma.pdf(time, 6) # 6-second peak
        undershoot = gamma.pdf(time, 12)# 12-sec undershoot
        hrf_vals = peak-0.35*undershoot
        return(hrf_vals)
    
    tr_timing = np.arange(0,duration,TR)
    hrf_tr = hrf(tr_timing)
    
    ratings = read_csv(ratings_file, index_col=0)
    regress_matrix = np.zeros((len(ratings),2))
    regress_matrix[:,0] = np.convolve(ratings['negative'],hrf_tr)[:len(ratings)]
    regress_matrix[:,1] = np.convolve(ratings['positive'],hrf_tr)[:len(ratings)]
    
    design_file_name = 'matrix.txt'
    np.savetxt(design_file_name, regress_matrix)
    design_file = abspath(design_file_name)
    
    return(design_file)

In [ ]:
# create the design file
make_design = Node(Function(input_names=['ratings_file', 'TR', 'duration'], 
                            output_names=['design_file'],
                            function=affectiveTiming), name='make_design')
make_design.inputs.TR = TR
make_design.inputs.duration = duration
make_design.inputs.ratings_file = affect_ratings

#run the GLM
estmodel = Node(GLM(dat_norm = True,
                    out_file = 'betas.nii.gz', 
                    out_cope='cope.nii.gz',
                    out_t_name = 'tstat.nii.gz', 
                    mask=gm_mask), 
                name= 'estmodel')

# split the copes
split_copes = Node(Split(dimension='t'),name='split_copes')

In [ ]:
L1workflow = Workflow(name='L1workflow_univariate')
L1workflow.connect([(infosource,selectfiles,[('subjid','subjid')]),
                    (selectfiles,estmodel,[('preproc_func','in_file')]),
                    
                    (make_design, estmodel, [('design_file','design')]),
                    (estmodel, split_copes, [('out_cope','in_file')]),
                    (split_copes, datasink, [('out_files','copes')]),
                    (estmodel, datasink, [('out_t','tstats')]),
                    (estmodel, datasink, [('out_file','betas')])
                   ])
L1workflow.base_dir = workflow_dir
#L1workflow.write_graph(graph2use='flat')
L1workflow.run('MultiProc', plugin_args={'n_procs': 6, 'memory_gb':10})

# Mixed Effect Linear Model

In [ ]:
def melm_analysis(merged_copes,subject_info,sample_template_mask, analysis):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    import statsmodels.formula.api as smf
    import numpy as np
    from pandas import DataFrame, read_csv, Series, concat
    from nilearn.masking import apply_mask, unmask
    from os.path import abspath
    from warnings import filterwarnings

    # Load the brain data
    sample_data = apply_mask(merged_copes, sample_template_mask)
    nterms = 3
    
    # Preallocate the output arrays
    pvals_data = np.zeros((nterms,sample_data.shape[1])).astype(float)
    coeffs_data = np.zeros((nterms,sample_data.shape[1])).astype(float)
    tstat_data = np.zeros((nterms,sample_data.shape[1])).astype(float)

    # define the mixed effects model
    #formula = 'activation ~ age_cent + {0} + age_cent*{0}'.format(analysis)
    formula = 'activation ~ age_cent'

    for x in range(0,sample_data.shape[1]):
        subject_info['activation'] = sample_data[:,x]
        model = smf.mixedlm(formula, subject_info, groups=subject_info['male'])
        try:
            fitted_model = model.fit()
            filterwarnings("ignore")
        except: 
            pass
        else:
            pvals_data[:,x] = np.array(fitted_model.pvalues)
            coeffs_data[:,x] = np.array(fitted_model.params)
            tstat_data[:,x] = np.array(fitted_model.tvalues)

    #save the voxel-wise results as niftis
    pvals_image = unmask(pvals_data, sample_template_mask)
    pvals_image.to_filename('{0}_pvalues.nii.gz'.format(analysis))
    coeff_image = unmask(coeffs_data, sample_template_mask)
    coeff_image.to_filename('{0}_coefficients.nii.gz'.format(analysis))
    tstat_image = unmask(tstat_data, sample_template_mask)
    tstat_image.to_filename('{0}_tstats.nii.gz'.format(analysis))

    pvals_file = abspath('{0}_pvalues.nii.gz'.format(analysis))
    coeff_file = abspath('{0}_coefficients.nii.gz'.format(analysis))
    tstat_file = abspath('{0}_tstats.nii.gz'.format(analysis))
    
    return(pvals_file,coeff_file,tstat_file)
        
def extract_cluster_betas(cluster_index_file, sample_betas, subject_ids):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from pandas import DataFrame, Series
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath, basename
    from nilearn.input_data import NiftiLabelsMasker
    
    subject_ids = sorted(subject_ids)
    
    ind_filename = basename(cluster_index_file) 
    out_prefix = ind_filename[:-7]
    masker = NiftiLabelsMasker(labels_img=cluster_index_file)
    betas = DataFrame(masker.fit_transform(sample_betas), index=subject_ids)
    
    betas.to_csv(out_prefix+'_extracted_values.csv')
    extracted_betas_csv = abspath(out_prefix+'_extracted_values.csv')
    
    return(extracted_betas_csv)

def get_cluster_peaks(clusters_file, stat_file):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, unravel_index, max
    from os.path import abspath
    
    # load up clusters
    clusters_nii = load(clusters_file)
    clusters_data = clusters_nii.get_data()
    cluster_labels, cluster_sizes = unique(clusters_data, return_counts=True)
    cluster_sizes = cluster_sizes[cluster_labels>0]
    cluster_labels = cluster_labels[cluster_labels>0]
    
    # set up dataframe
    cluster_info = DataFrame(columns=['clust_num','peak','num_voxels','X','Y','Z'])
    cluster_info['clust_num'] = Series(cluster_labels,index=None)
    
    for i in range(0,len(cluster_labels)):
        # load up stat image
        stat_nii = load(stat_file)
        stat_data = stat_nii.get_data()
        stat_data[clusters_data!=cluster_labels[i]]=0
        location=unravel_index(stat_data.argmax(), stat_data.shape)
        cluster_info.iloc[i,0]=cluster_labels[i]
        cluster_info.iloc[i,1]=max(stat_data)
        cluster_info.iloc[i,2]=cluster_sizes[i]
        cluster_info.iloc[i,3]=location[0]
        cluster_info.iloc[i,4]=location[1]
        cluster_info.iloc[i,5]=location[2]
    
    out_prefix = clusters_file[:-7]
    cluster_info.to_csv(out_prefix + '_peaks.csv')
    cluster_info_file = abspath(out_prefix + '_peaks.csv')
    return(cluster_info_file)

In [ ]:
# Sink data of interest 
substitutions = [('_condition_0000', 'negative'),
                 ('_condition_0001','positive')] #output file name substitutions
datasink = Node(DataSink(base_directory = secondlevel_dir,
                        container = secondlevel_dir,
                        substitutions = substitutions), 
                name='datasink')

# select parameter estimates
cope_template = {'beta_maps':firstlevel_dir + '/copes/*/vol%s.nii.gz'}
betamap_grabber = Node(DataGrabber(sort_filelist=True,
                                   field_template = cope_template,
                                   base_directory=firstlevel_dir,
                                   template=firstlevel_dir + '/copes/*/vol%s.nii.gz',
                                   infields=['condition'],
                                   template_args={'beta_maps':[['condition']]}), 
                       name='betamap_grabber')
betamap_grabber.iterables=('condition',['0000','0001'])

# merge individual parameter estimates together
merge_copes = Node(Merge(dimension='t',tr=TR),name='merge_copes')

# Run MELM
melm = Node(Function(input_names=['merged_copes','subject_info','sample_template_mask','analysis'],
                     output_names=['pvals_file','coeff_file','tstat_file'], 
                     function=melm_analysis), name='melm')
melm.inputs.sample_template_mask = gm_mask
melm.inputs.subject_info = subject_info
melm.iterables=('analysis',['age'])
#melm.iterables=('analysis',['NEG','SUR','EC'])

# create invert t-stats
inv_tstats = Node(MathsCommand(args='-mul -1',out_file='tstats_inv.nii.gz'),name='inv_tstats')

# split tstats
split_tstats = Node(Split(dimension='t'), name='split_tstats')

#split inv tstats
split_invtstats = Node(Split(dimension='t'), name='split_invtstats')

# cluster results
clust_tstats = MapNode(Cluster(threshold=3.12, dlh=2.57, out_index_file='clusters.nii.gz'), 
                       name='clust_tstats', iterfield=['in_file'])

# cluster inverted results
clust_invtstats = MapNode(Cluster(threshold=3.12, dlh=2.57, out_index_file='clusters.nii.gz'), 
                          name='clust_invtstats', iterfield=['in_file'])

# extract PEs from clusters
extract_betas = MapNode(Function(input_names=['cluster_index_file', 'sample_betas', 'subject_ids'], 
                                 output_names=['extracted_betas_csv'], 
                                 function=extract_cluster_betas), 
                        name='extract_betas', iterfield=['cluster_index_file'])
extract_betas.inputs.subject_ids = subjects_list

# extract PEs from inverted clusters
extract_invbetas = MapNode(Function(input_names=['cluster_index_file', 'sample_betas', 'subject_ids'], 
                                    output_names=['extracted_betas_csv'], 
                                    function=extract_cluster_betas), 
                           name='extract_invbetas', iterfield=['cluster_index_file'])
extract_invbetas.inputs.subject_ids = subjects_list

# extract peaks
extract_peaks = MapNode(Function(input_names=['clusters_file', 'stat_file'], 
                                 output_names=['cluster_info_file'], 
                                 function=get_cluster_peaks), 
                        name='extract_peaks', iterfield=['clusters_file','stat_file'])

# extract inverted peaks
extract_invpeaks = MapNode(Function(input_names=['clusters_file', 'stat_file'],
                                    output_names=['cluster_info_file'], 
                                    function=get_cluster_peaks), 
                           name='extract_invpeaks', iterfield=['clusters_file','stat_file'])

In [ ]:
melm_workflow = Workflow(name='melm_workflow_age')
melm_workflow.connect([(betamap_grabber, merge_copes,[('beta_maps','in_files')]),
                       (merge_copes, melm, [('merged_file','merged_copes')]),
                       (melm, inv_tstats, [('tstat_file','in_file')]),
                       (melm, split_tstats, [('tstat_file','in_file')]),
                       (inv_tstats, split_invtstats, [('out_file','in_file')]),
                       (split_tstats, clust_tstats, [('out_files','in_file')]),
                       (split_invtstats, clust_invtstats, [('out_files','in_file')]),
                       (merge_copes, extract_betas, [('merged_file','sample_betas')]),
                       (merge_copes, extract_invbetas, [('merged_file','sample_betas')]),
                       (clust_tstats, extract_betas, [('index_file','cluster_index_file')]),
                       (clust_invtstats, extract_invbetas, [('index_file','cluster_index_file')]),
                       (clust_tstats, extract_peaks, [('index_file','clusters_file')]),
                       (clust_invtstats, extract_invpeaks, [('index_file','clusters_file')]),
                       (split_tstats, extract_peaks, [('out_files','stat_file')]),
                       (split_invtstats, extract_invpeaks, [('out_files','stat_file')]),
                       (melm, datasink, [('pvals_file','pvals'),
                                         ('coeff_file','coefficients'),
                                         ('tstat_file','tstats')]),
                       (clust_tstats, datasink, [('index_file','positive_clusters')]),
                       (clust_invtstats, datasink, [('index_file','negative_clusters')]),
                       (extract_betas, datasink, [('extracted_betas_csv','extracted_betas')]),
                       (extract_invbetas, datasink, [('extracted_betas_csv','extracted_inv_betas')]),
                       (extract_invpeaks, datasink, [('cluster_info_file','cluster_inv_tables')]),
                       (extract_peaks, datasink, [('cluster_info_file','cluster_tables')])
                      ])
melm_workflow.base_dir = workflow_dir
#melm_workflow.write_graph(graph2use='flat')
melm_workflow.run('MultiProc', plugin_args={'n_procs': 6, 'memory_gb':32})